# Introduction to Geopandas

Geopandas (http://geopandas.org/) makes it possible to work with geospatial data in Python in a relatively easy way. Geopandas combines the capabilities of the data analysis library [pandas](https://pandas.pydata.org/pandas-docs/stable/) with other packages like [shapely](https://shapely.readthedocs.io/en/stable/manual.html) and [fiona](https://fiona.readthedocs.io/en/latest/manual.html) for managing spatial data. 

The main data structures in geopandas; `GeoSeries` and `GeoDataFrame` extend the capabilities of `Series` and `DataFrames` from pandas. This means that we can apply our pandas skills also with geopandas data structures. If you need to refresh your memory about pandas, check out week 5 and 6 lesson materials from the [Geo-Python website](geo-python.github.io). 

The main difference between geodataframes and pandas dataframes is that [a geodataframe should contain one column for geometries](http://geopandas.org/data_structures.html#geodataframe) (by default, the name of this column is `'geometry'`). The geometry column is a [geoseries](http://geopandas.org/data_structures.html#geoseries) which contains the geometries (points, lines, polygons, multipolygons etc.) for each row of data. 

As we learned in the Geo-Python course, it is conventional to import pandas as `pd`. Similarly,we will import geopandas as `gpd`:

In this lesson, we will cover basics steps needed for interacting with spatial data in Python using geopandas:

- Managing filepaths
- Reading a shapefile 
- Geometry calculations 
- Writing a shapefile
- Grouping and splitting spatial data into multiple layers

Before diving deeper into geopandas functionalities, let's first acquire some input data to work with. 

## Input data: Finnish topographic database 

- [Topographic Database from the National Land Survey of Finland (NLS)](https://www.maanmittauslaitos.fi/en/maps-and-spatial-data/expert-users/product-descriptions/topographic-database). 
- The data set is licensed under the NLS' [open data licence](https://www.maanmittauslaitos.fi/en/opendata-licence-cc40) (CC BY 4.0).
- Structure of the data is described in a separate Excel file ([download link](http://www.maanmittauslaitos.fi/sites/maanmittauslaitos.fi/files/attachments/2018/10/maastotietokanta_kohdemalli_eng.xlsx)).
- Further information about file naming at [fairdata.fi](https://etsin.fairdata.fi/dataset/5023ecc7-914a-4494-9e32-d0a39d3b56ae).
- We acquired the data via the [CSC open data portal](https://avaa.tdata.fi/web/paituli/latauspalvelu):

![Paituli data download](img/Paituli_maastotietokanta_download.png)


In this lesson, we will focus on **terrain objects** (Feature group: "Terrain/1") downloaded as Shapefiles. According to the [naming convention of the Topographic Database](https://etsin.fairdata.fi/dataset/5023ecc7-914a-4494-9e32-d0a39d3b56ae) for Shapefiles, all files that start with a letter `m` and end with `p` contain the objects we are interested in (Terrain/1 polygons). Furthermore, the Terrain/1 feature group contains several feature classes. **Our final task in this lesson is to save all these feature classes into separate files**.

*Terrain/1 features in the Topographic Database:*

|  feature class | Name of feature                                            | Feature group |
|----------------|------------------------------------------------------------|---------------|
| 32421          | Motor traffic area                                         | Terrain/1     |
| 32200          | Cemetery                                                   | Terrain/1     |
| 34300          | Sand                                                       | Terrain/1     |
| 34100          | Rock - area                                                | Terrain/1     |
| 34700          | Rocky area                                                 | Terrain/1     |
| 32500          | Quarry                                                     | Terrain/1     |
| 32112          | Mineral resources extraction area, fine-grained material   | Terrain/1     |
| 32111          | Mineral resources extraction area, coarse-grained material | Terrain/1     |
| 32611          | Field                                                      | Terrain/1     |
| 32612          | Garden                                                     | Terrain/1     |
| 32800          | Meadow                                                     | Terrain/1     |
| 32900          | Park                                                       | Terrain/1     |
| 35300          | Paludified land                                            | Terrain/1     |
| 35412          | Bog, easy to traverse forested                             | Terrain/1     |
| 35411          | Open bog, easy to traverse treeless                        | Terrain/1     |
| 35421          | Open fen, difficult to traverse treeless                   | Terrain/1     |
| 33000          | Earth fill                                                 | Terrain/1     |
| 33100          | Sports and recreation area                                 | Terrain/1     |
| 36200          | Lake water                                                 | Terrain/1     |
| 36313          | Watercourse area                                           | Terrain/1     |

## Downloading data

On Binder and CSC Notebook environment, you can use `wget` program to download the data from the command line. Let's download the data ([download link](https://github.com/AutoGIS/data/raw/master/L2_data.zip)) into the same folder with the lesson 2 notebooks (`.../notebooks/L2`):

- Open up a new terminal window
- Navigate to the correct folder in the terminal:

```
# Navigate to lesson 2 notebooks directory:
$ cd /home/jovyan/work/autogis/notebooks/notebooks/L2
    
```
- use `wget` utility to dowload the data from the dowload link:
    
```
$ wget https://github.com/AutoGIS/data/raw/master/L2_data.zip
    
```
<div class="alert alert-info">

**Copy-paste**
    
You can copy/paste things to JupyterLab Terminal by pressing `SHIFT` + `RIGHT-CLICK` on your mouse and choosing `Paste`.

</div>

Once you have downloaded the `L2_data.zip` file into your home directory, you can unzip the file using `unzip` command in the Terminal (or e.g. 7zip on Windows if working with own computer). Run the following commands in the `.../notebooks/L2` -directory:

``` 
$ unzip L2_data.zip
$ ls L2_data

```
You can also check the contents of the downloaded and unzipped file in the file browser window. 

The L2_data folder contains several subfolders. After unzipping the downloaded file, you can find the data for this tutorial under: `L2_data/NLS/2018/L4/L41/L4132R.shp`. Notice that Shapefile -fileformat contains many separate files such as `.dbf` that contains the attribute information, and `.prj` -file that contains information about coordinate reference system.

## Managing filepaths

Built-in module `os` provides many useful functions for interacting with the operating system. One of the most useful submodules in the os package is the [os.path-module](https://docs.python.org/2/library/os.path.html) for manipulating file paths. This week, we have data in different sub-folders and we can practice how to use `os` path tools when defining filepaths.

- Let's import `os` and see how we can construct a filepath by joining a folder path and file name:

## Reading a Shapefile

Typically reading the data into Python is the first step of the analysis pipeline. There are various different GIS data formats available. [Shapefile](https://en.wikipedia.org/wiki/Shapefile), [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON), [KML](https://en.wikipedia.org/wiki/Keyhole_Markup_Language), and [GPKG](https://en.wikipedia.org/wiki/GeoPackage) are one of the most common vector data formats currently in use. [Geopandas](http://geopandas.org/io.html) is capable of reading data from all of these formats (plus many more). 

In geopandas, we use a generic function [.from_file()](http://geopandas.org/reference.html#geopandas.GeoDataFrame.to_file) for reading in different data formats. In the bacground, Geopandas uses [fiona.open()](https://fiona.readthedocs.io/en/latest/fiona.html#fiona.open) when reading in data.

- When reading in a Shapefile, we only need to pass the filepath when reading data:

- Let's see check the data type:

Here we see that our `data` -variable is a `GeoDataFrame`. GeoDataFrame extends the functionalities of
`pandas.DataFrame` in a way that it is possible to handle spatial data using similar approaches and datastructures as in pandas (hence the name geopandas). 

- Let's take a look at our data and print the first rows using the `head()` -function:

- Check all column names:

As most of you probably notice, all the column names are in Finnish... 
- Let's select only the useful columns and rename them into English:

In [ ]:
data = data[['RYHMA', 'LUOKKA',  'geometry']]

- Define new column names in a dictionary:

In [ ]:
colnames = {'RYHMA':'GROUP', 'LUOKKA':'CLASS'}

- rename the columns:

<div class="alert alert-info">

**Task**
    
Figure out the following information from our input data based on your pandas-skills from the Geo-Python course:
    
- Number of rows?
- Number of classes?
- Number of groups?
</div>

Solutions:

It is always a good idea to explore your data also on a map. Creating a simple map from a `GeoDataFrame` is really easy: you can use ``.plot()`` -function from geopandas that **creates a map based on the geometries of the data**. Geopandas actually uses matplotlib for plotting which we introduced in [Lesson 7 of the Geo-Python course](https://geo-python.github.io/site/notebooks/L7/matplotlib.html).

- Let's try it out, and plot our GeoDataFrame:

In [ ]:
%matplotlib inline


Voilá! As we can see, it is really easy to produce a map out of your Shapefile with geopandas. Geopandas automatically positions your map in a way that it covers the whole extent of your data.

*If you are living in the Helsinki region, you might recognize the shapes plotted on the map!*

## Geometries in Geopandas

Geopandas takes advantage of Shapely's geometric objects. Geometries are stored in a column called *geometry* that is a default column name for
storing geometric information in geopandas.

-  Let's print the first 5 rows of the column 'geometry':

As we can see the `geometry` column contains familiar looking values, namely Shapely `Polygon` -objects. Since the spatial data is stored as Shapely objects, **it is possible to use all of the functionalities of the Shapely module** when dealing with geometries in geopandas.

Let's have a closer look at the polygons and try to apply some of the [Shapely methods we learned last week](https://automating-gis-processes.github.io/site/notebooks/L1/geometric-objects.html#Polygon). 

- Let's start by checking the area of the first polygon in the data:



Let's do the same for the first five rows in the data; 

- Iterate over the GeoDataFrame rows using the `iterrows()` -function that we learned [during the Lesson 6 of the Geo-Python course](https://geo-python.github.io/site/notebooks/L6/pandas/advanced-data-processing-with-pandas.html#Iterating-rows-and-using-self-made-functions-in-Pandas).
- For each row, print the area of the polygon (here, we'll limit the for-loop to a selection of the first five rows):

In [ ]:
# Iterate over rows and print the area of a Polygon
for index, row in data[0:5].iterrows():
    
    # Get the area from the shapely-object stored in the geometry-column


As you see from here, all the functionalities of **pandas**, such as the `iterrows()` function, are directly available in Geopandas without the need to call pandas separately because Geopandas is an **extension** for pandas. 

In practice, it is not necessary to use the iterrows()-approach to calculate the area for all features. Geodataframes and geoseries have an attribute `area` which we can use for accessing the area for each feature at once: 

- Let's next create a new column into our GeoDataFrame where we calculate and store the areas of individual polygons:

- Check the output:

These values correspond to the ones we saw in previous step when iterating rows.

- Let's check what is the `min`, `max` and `mean` of those areas using familiar functions from our previous Pandas lessions.


In [ ]:
# Maximum area


In [ ]:
# Minimum area


In [ ]:
# Mean area


## Writing a shapefile

It is possible to export GeoDataFrames into various data formats using [gpd.to_file()](http://geopandas.org/io.html#writing-spatial-data). Here, we will first learn how to export a subset of the data into a Shapefile.

- Let's first select one class (class number `36200`, "Lake water") from the data as a new GeoDataFrame:


- Check the selection:

- write this layer into a new Shapefile using the `gpd.to_file()` -function:

In [ ]:
# Create a output path for the data

In [ ]:
# Write data to file

<div class="alert alert-info">

**Task**
    
Read the output Shapefile in a new geodataframe, and check that the data looks ok.
</div>

## Grouping the Geodataframe

One really useful function that can be used in Pandas/Geopandas is [.groupby()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) which groups data based on values on selected column(s). We saw and used this function already in [Lesson 6 of the Geo-Python course](https://geo-python.github.io/2018/notebooks/L6/pandas/advanced-data-processing-with-pandas.html#Aggregating-data-in-Pandas-by-grouping). 

Next we will automate the file export task; we will group the data based on column `CLASS` and export a shapefile for each class.

Let's continue with the same input file we already read previously into the variable `data`. We also selected and renamed a subset of the columns.

- Check the first rows of the data:

The `CLASS` column in the data contains information about different land use types. With `.unique()` -function we can quickly see all different values in that column:

In [ ]:
# Print all unique values in the column 'CLASS'


- Now we can use that information to group our data and save all land use types into different layers:

In [ ]:
# Group the data by class


As we can see, `groupby` -function gives us an object called `DataFrameGroupBy` which is similar to list of keys and values (in a dictionary) that we can iterate over. For more information about grouped objects, see [Lesson 6 of the Geo-Python course](https://geo-python.github.io/2018/notebooks/L6/pandas/advanced-data-processing-with-pandas.html#Aggregating-data-in-Pandas-by-grouping).

- Check out all group keys:

The group keys are unique values from the column by which we grouped the dataframe.

- Check how many rows of data each group has:

There are, for example, 56 lake polygons in the input data.

We can also check how the _last_ group looks like (we have the variables in memory from the last iteration of the for-loop):

Notice that the index numbers refer to the row numbers in the original data -GeoDataFrame.

Check also the data type of the group:

As we can see, each set of data are now grouped into separate GeoDataFrames, and we can save them into separate files.

### Saving multiple output files

Let's **export each class into a separate Shapefile**. While doing this, we also want to **create unique filenames for each class**.

When looping over the grouped object, information about the class is stored in the variable `key`, and we can use this information for creating new variable names inside the for-loop. For example, we want to name the shapefile containing lake polygons as "terrain_36200.shp".


<div class="alert alert-info">

**String formatting**
    
There are different approaches for formatting strings in Python. Here are a couple of different ways for putting together file-path names using two variables:

```
basename = "terrain"
key = 36200

# OPTION 1. Concatenating using the `+` operator:
out_fp = basename + "_" + str(key) + ".shp"

# OPTION 2. Positional formatting using `%` operator
out_fp = "%s_%s.shp" %(basename, key)
    
# OPTION 3. Positional formatting using `.format()`
out_fp = "{}_{}.shp".format(basename, key)
```
    
Read more from here: https://pyformat.info/
</div>


Let's now export terrain classes into separate Shapefiles.

- First, create a new folder for the outputs:

In [ ]:
# Determine output directory
output_folder = r"L2_data/"

# Create a new folder called 'Results' 
result_folder = os.path.join(output_folder, 'Results')

# Check if the folder exists already
if not os.path.exists(result_folder):
    # If it does not exist, create one
    os.makedirs(result_folder)

At this point, you can go to the file browser and check that the new folder was created successfully.

- Iterate over groups, create a file name, and save group to file:

In [ ]:
# Iterate over the groups
for key, group in grouped:
    
    # Format the filename 


    # Print information about the process


    # Create an output path


    # Export the data

Excellent! Now we have saved those individual classes into separate Shapefiles and named the file according to the class name. These kind of grouping operations can be really handy when dealing with layers of spatial data. Doing similar process manually would be really laborious and error-prone.

### Extra: save data to csv

We can also extract basic statistics from our geodataframe, and save this information as a text file. 

Let's summarize the total area of each group:

- save area info to csv using pandas:

## Summary

In this tutorial we introduced the first steps of using geopandas. More specifically you should know how to:

1. Read data from Shapefile using geopandas

2. Access geometry information in a geodataframe

4. Write GeoDataFrame data from Shapefile using geopandas

5. Automate a task to save specific rows from data into Shapefile based on specific key using `groupby()` -function

6. Extra: saving attribute information to a csv file.


 